### Model creation and training code for small randomforest model on iris datset

In [ ]:
import json
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as Rf
import torch
import ezkl
import os
from hummingbird.ml import convert

iris = load_iris()
X, y = iris.data, iris.target
X = X.astype(np.float32)
X_train, X_test, y_train, y_test = train_test_split(X, y)
clr = Rf()
clr.fit(X_train, y_train)



torch_rf = convert(clr, 'torch')

diffs = []
for i in range(len(X_test)):
    torch_pred = torch_rf.predict(torch.tensor(X_test[i].reshape(1, -1)))
    sk_pred = clr.predict(X_test[i].reshape(1, -1))
    diffs.append(torch_pred[0].round() - sk_pred[0])

print("num diffs", sum(diffs))


In [ ]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

### Exporting model 

In [ ]:
shape = X_train.shape[1:]
x = torch.rand(1, *shape, requires_grad=False)
torch_out = torch_rf.predict(x)
# Export the model
torch.onnx.export(torch_rf.model,               # model being run
                  # model input (or a tuple for multiple inputs)
                  x,
                  # where to save the model (can be a file or file-like object)
                  "network.onnx",
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=11,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names=['input'],   # the model's input names
                  output_names=['output'],  # the model's output names
                  dynamic_axes={'input': {0: 'batch_size'},    # variable length axes
                                'output': {0: 'batch_size'}})

d = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_shapes=[shape],
            input_data=[d],
            output_data=[o.reshape([-1]).tolist() for o in torch_out])

# Serialize data into file:
json.dump(data, open("input.json", 'w'))

### Generating settings for circuit generation on the basis of model

In [ ]:
run_args = ezkl.PyRunArgs()
run_args.logrows = 20

res = ezkl.gen_settings(model_path, settings_path, py_run_args=run_args)
assert res == True


### calibrating the circuit setitngs wiht calibration on the bsis of data ranges

In [ ]:
cal_path = os.path.join("calibration.json")

data_array = (torch.rand(20, *shape, requires_grad=True).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

json.dump(data, open(cal_path, 'w'))


await ezkl.calibrate_settings(cal_path, model_path, settings_path, "resources")

### Circuit compilation

In [ ]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

### srs path

In [ ]:

res = await ezkl.get_srs( settings_path)

### now generate the witness file 

In [ ]:


res = await ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

### HERE WE SETUP THE CIRCUIT PARAMS
### WE GOT KEYS
### WE GOT CIRCUIT PARAMETERS
### EVERYTHING we need FOR ZKP

In [ ]:


res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

### GENERATE A PROOF

In [ ]:
proof_path = os.path.join('proof.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

### VERIFY proof off-chain 

In [ ]:


res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
    )

assert res == True
print("verified")